## Bayesian Optimization with Custom Objectives
In this tutorial we demonstrate the use of Xopt to preform Bayesian Optimization on
custom objectives. In this case, we develop models of individual components of the
objective function and combine samples from these models to calculate predicted
objective values.

In this example we try to maximize the objective function
$$f(g_1(x),g_2(x)) = \min(g_1(x), g_2(x))$$ where $g_1(x) = (x-0.5)^2$ and $g_2(x) =
(x - 2)^2$.


## Define the test problem

In [ ]:
from xopt.vocs import VOCS

import torch

from xopt.evaluator import Evaluator
from xopt.generators.bayesian import ExpectedImprovementGenerator
from xopt import Xopt
from xopt.generators.bayesian.objectives import CustomXoptObjective
from torch import Tensor
from typing import Optional


# define variables and function objectives
vocs = VOCS(variables={"x": [0.0, 2.0]}, observables=["g1", "g2"])

In [ ]:
# define a test function to optimize


def sin_function(input_dict):
    return {"g1": (input_dict["x"]) ** 2, "g2": (input_dict["x"] - 2.0) ** 2}

## Create Xopt objects
Create the evaluator to evaluate our test function and create a generator that uses
the Upper Confidence Bound acquisition function to perform Bayesian Optimization.

In [ ]:
class MyObjective(CustomXoptObjective):
    def forward(self, samples: Tensor, X: Optional[Tensor] = None) -> Tensor:
        return torch.min(
            samples[..., self.vocs.output_names.index("g1")],
            samples[..., self.vocs.output_names.index("g2")],
        )


evaluator = Evaluator(function=sin_function)
generator = ExpectedImprovementGenerator(
    vocs=vocs,
    custom_objective=MyObjective(vocs),
    log_transform_acquisition_function=True,
)
X = Xopt(evaluator=evaluator, generator=generator, vocs=vocs)
print(X)

X.random_evaluate(2)

n_steps = 5

# test points for plotting
test_x = torch.linspace(*X.vocs.bounds.flatten(), 50).double()

for i in range(n_steps):
    # get the Gaussian process model from the generator
    model = X.generator.train_model()

    X.generator.visualize_model()

    # do the optimization step
    X.step()

In [ ]:
# access the collected data
X.data

## Getting the optimization result
To get the best point (without evaluating it) we ask the generator to
predict the optimum based on the posterior mean.

In [ ]:
X.generator.get_optimum()

## Customizing optimization
Each generator has a set of options that can be modified to effect optimization behavior

In [ ]:
X.generator.dict()